## 数据分析

In [ ]:
import json
from collections import defaultdict
with open('../data/NYT/train_triples.json', 'r', encoding='utf-8') as f_tr, \
    open('../data/NYT/val_triples.json', 'r', encoding='utf-8') as f_val, \
    open('../data/NYT/test_triples.json', 'r', encoding='utf-8') as f_te:
    d_tr = json.load(f_tr)
    d_val = json.load(f_val)
    d_te = json.load(f_te)

In [ ]:
d_all = d_tr+d_val+d_te

In [ ]:
len(d_all)

**平均关系数和主体数**

WebNLG*:

平均关系数：2.23 --> 3, 平均主体数：1.47 --> 2

NYT*:

平均关系数：1.44 --> 2, 平均主体数：1.25 --> 2

In [ ]:
d_all

In [ ]:
data_tmp = []
for sample in d_all:
    rel_dict = defaultdict(list)
    for triple in sample['triple_list']:
        rel_dict[triple[1]].append((triple[0], triple[-1]))
    data_tmp.append(rel_dict)

In [ ]:
data_tmp = []
for sample in d_all:
    rel_dict = defaultdict(list)
    for triple in sample['triple_list']:
        rel_dict[triple[0]].append((triple[0], triple[-1]))
    data_tmp.append(rel_dict)

In [ ]:
data_tmp

In [ ]:
num_rel = [len(d) for d in data_tmp]

In [ ]:
sum(num_rel)/len(num_rel)

In [ ]:
text_length = [len(sample['text'].split(' ')) for sample in d_all]

max_len = 80

In [ ]:
import pandas as pd
pd.Series(text_length).describe()

In [ ]:
len(d_all)

In [ ]:
import sys
sys.path.append('..')
from utils import Params
from dataloader import CustomDataLoader
params = Params()
datalodaer = CustomDataLoader(params)

In [ ]:
data_len = []
for d in d_all:
    text = d['text']
    data_len.append(len(datalodaer.tokenizer.tokenize(text)))

In [ ]:
import pandas as pd
pd.Series(data_len).describe([0.99])

## case study

In [ ]:
import pandas as pd
df = pd.read_csv('./test_result.csv',index_col=0)

In [ ]:
df.head()

In [ ]:
df.to_excel('./result.xlsx')

original version

In [ ]:
import sys
sys.path.append('..')
from utils import Params
from dataloader import CustomDataLoader
params = Params()
dataloader = CustomDataLoader(params)

In [ ]:
def apply_func(row, sign='pre'):
    token_text = dataloader.tokenizer.tokenize(row.text)
    new = []
    if sign=='pre':
        iter_ = row.pre
    else:
        iter_ = row.truth
    for triple in eval(iter_):
        sub = triple[0]
        obj = triple[1]
        re = triple[-1]
        sub_t = token_text[sub[1]:sub[-1]]
        obj_t = token_text[obj[1]:obj[-1]]
        new.append((sub_t, obj_t, re, sub[1], obj[1]))
    new = sorted(new, key=lambda x: (x[-2], x[-1]))
    return new

In [ ]:
df['true_label'] = df.apply(apply_func, axis=1)

In [ ]:
df['pre_label'] = df.apply(apply_func, axis=1)

In [ ]:
df = df.drop(['pre', 'truth'], axis=1)

In [ ]:
df